In [24]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [25]:
base = pd.read_csv('iec6100429.csv')

In [26]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [27]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape, x.shape

((15,), (15, 191))

In [28]:
y = y.values.reshape(-1,1)
y.shape

(15, 1)

In [29]:
y = onehot.fit_transform(y).toarray()

In [30]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.3)

In [31]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((15, 191), (5, 191), (15, 3), (5, 3))

In [32]:
neuronios_entrada = x.shape[1]
neuronios_entrada

191

In [54]:
neuronios_oculta1 = 10
neuronios_oculta1

10

In [55]:
neuronios_saida = y.shape[1]
neuronios_saida

3

In [56]:
# 100 -> 51 -> 3

In [57]:
w = {'oculta1': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta1])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta1, neuronios_saida]))}

In [58]:
b = {'oculta1': tf.Variable(tf.random_normal([neuronios_oculta1])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}

In [59]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [60]:
def mlp(x, w, bias):
    camada_oculta1 = tf.nn.relu(tf.add(tf.matmul(x, w['oculta1']), b['oculta1']))
    camada_saida = tf.add(tf.matmul(camada_oculta1, w['saida']), b['saida'])
    return camada_saida

In [61]:
modelo = mlp(xph, w, b)

In [62]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.3).minimize(erro)

In [63]:
batch_size = 10
batch_total = int(len(x_treinamento) / batch_size)
batch_total, x_treinamento.shape

(1, (10, 191))

In [64]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(1000):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 100 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])
    
    print('fim do treinamento')
    
    previsoes = tf.nn.softmax(mlp(xph, w_final, b_final))
    previsoes_corretas = tf.equal(tf.argmax(previsoes, 1), tf.argmax(yph, 1))
    taxa_acerto = tf.reduce_mean(tf.cast(previsoes_corretas, tf.float32))
    print(sess.run(taxa_acerto, feed_dict = {xph: x_teste, yph: y_teste}))

Época: 1 erro: 34.65637969970703
Época: 101 erro: 1.0296581983566284
Época: 201 erro: 1.0296529531478882
Época: 301 erro: 1.0296530723571777
Época: 401 erro: 1.0296529531478882
Época: 501 erro: 1.0296529531478882
Época: 601 erro: 1.0296529531478882
Época: 701 erro: 1.0296529531478882
Época: 801 erro: 1.0296529531478882
Época: 901 erro: 1.0296529531478882
fim do treinamento
0.2
